In [ ]:
import cv2 as cv
import numpy as np

In [ ]:
from evaluation.config import *

time_config = TimingConfig.load_json("logs/time_config.json")

In [ ]:
from evaluation.analysis import Plotter
plt = Plotter("logs/bboxes.csv", time_config)
plt.plot_speed_hist()
plt.plot_area_vs_speed(min_speed=0.)

In [ ]:
cv.destroyAllWindows()
raise Exception("Finished")

In [ ]:
from evaluation.simulator import *
from evaluation.sim_controllers import *
from utils.path_utils import Files
from evaluation.config import *

experiment_config = ExperimentConfig(
    name="exp1",
    num_frames=61200,
    frames_per_sec=60,
    orig_resolution=(1600, 1400),
    px_per_mm=90,
    init_position=(1300, 1200),
)

time_config = TimingConfig(
    frames_per_sec=60,
    imaging_time_ms=200,
    pred_time_ms=40,
    moving_time_ms=50,
    px_per_mm=90,
    camera_size_mm=(4, 4),
    micro_size_mm=(0.220, 0.210),
)

log_config = LogConfig(
    root_folder="logs/",
    save_mic_view=False,
    save_cam_view=False,
    save_err_view=False,
)

yolo_config = YoloConfig(
    model_path="yolov8m-trained.pt",
    verbose=True,
)

log_config.create_dirs()


time_config.save_json("logs/time_config.json")
log_config.save_json("logs/log_config.json")
yolo_config.save_json("logs/yolo_config.json") 
experiment_config.save_json("logs/experiment_config.json")


frames_path = "D:/Guy_Gilad/RawData/Exp_20240213_N2_JC01/"
files = Files(frames_path, extension="Bmp", sorting_key=lambda f: int(f.split("-")[-1].split(".")[0]))
files_list = [file for file in files]
reader = FrameReader(frames_path, files_list)

In [ ]:
print(time_config.imaging_frame_num + time_config.moving_frame_num)

In [ ]:
weights = np.asanyarray([0.0411 ,0.    , 0.    , 0.  ,   0.0003, 0.0802 ,0.081 , 0.4759, 0.8711]
)
sample_times = np.asanyarray(
    [
        0,
        5,
        9,
        -time_config.cycle_length + 0,
        -time_config.cycle_length + 5,
        -time_config.cycle_length + 9,
        -2 * time_config.cycle_length + 0,
        -2 * time_config.cycle_length + 5,
        -2 * time_config.cycle_length + 9,
    ]
)

In [ ]:
from evaluation.sim_controllers.test_controller1 import TestController1
from evaluation.sim_controllers.polyfit_controller import PolyfitController

sim_controller = YoloController(time_config, yolo_config)
#sim_controller = TestController1(time_config, "bboxes.csv")
#sim_controller = PolyfitController(time_config, "bboxes.csv", degree=1, weights=weights, sample_times=sample_times)
#sim_controller = OptimalController(time_config, "bboxes.csv")
#sim_controller = CsvController(time_config, "bboxes.csv")
log_controller = LoggingController(sim_controller, log_config)
sim = Simulator(time_config, experiment_config, log_controller, reader=reader)

In [ ]:
sim.run(visualize=False)

In [ ]:
cv.destroyAllWindows()
raise Exception("Finished")

In [ ]:
from frame_reader import FrameReader

reader = FrameReader.create_from_directory("Archive/")

In [ ]:
from dataset.create.box_calculator import BoxCalculator
from dataset.create.sample_extractor import SampleExtractor

bbox_calc = BoxCalculator(reader)
extractor = SampleExtractor(bbox_calc)
extractor.create_all_samples(target_size=(400, 400), save_folder="samples")

In [ ]:
from evaluation.sim_controllers.polyfit_controller import WeightEvaluator

# input_offsets = np.asanyarray([0, 3, 6, 9])

input_offsets = np.asanyarray(
    [
        0,
        5,
        9,
        -time_config.cycle_length + 0,
        -time_config.cycle_length + 5,
        -time_config.cycle_length + 9,
        -2 * time_config.cycle_length + 0,
        -2 * time_config.cycle_length + 5,
        -2 * time_config.cycle_length + 9,
    ]
)

start_times = np.arange(60000 // time_config.cycle_length) * time_config.cycle_length

evaluator = WeightEvaluator(
    "bboxes.csv",
    time_config,
    input_offsets=input_offsets,
    start_times=start_times,
    eval_offset= time_config.cycle_length + time_config.imaging_frame_num // 2,
    min_speed=0,
)

In [ ]:
def eval_func(weights: np.ndarray) -> float:
    return evaluator.eval(weights, deg=1)

In [ ]:
from mealpy.swarm_based.PSO import OriginalPSO
from mealpy.utils.problem import Problem
from mealpy.utils.termination import Termination
from mealpy.utils.agent import Agent
import mealpy

#optim = mealpy.ICA.OriginalICA()
optim = mealpy.PSO.OriginalPSO()

termination = Termination(max_epoch=300, max_fe=None, max_time=None, max_early_stop=100)

bounds = mealpy.FloatVar(lb=np.zeros(len(input_offsets)), ub=np.ones(len(input_offsets)))

problem = Problem(obj_func=eval_func, bounds=bounds, minimax="min")

best: Agent = optim.solve(
    problem,
    mode="single",
    n_workers=5,
    termination=termination,
)

In [ ]:
print(optim.g_best.target.fitness)
print(optim.g_best.solution / np.linalg.norm(best.solution))

In [ ]:
import numpy as np

arr = np.asanyarray(
    [
        [
            [np.nan, np.nan],
            [1, 1],
        ],
        [
            [2, 2],
            [1, 1],
        ],
    ]
)

print(arr.shape)

print(np.all(np.isfinite(arr), axis=1))